In [ ]:
from google.colab import files

uploaded = files.upload()

Saving X.npy to X.npy
Saving y.npy to y.npy


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random

In [ ]:
X = np.load('X.npy')
y = np.load('y.npy')

# Shuffle and split
indices = list(range(len(X)))
random.shuffle(indices)

X = X[indices]
y = y[indices]

split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

In [ ]:
class TicTacToeDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TicTacToeDataset(X_train, y_train)
val_dataset = TicTacToeDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

In [ ]:
class TicTacToeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(9, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 9)
        )

    def forward(self, x):
        return self.model(x)

model = TicTacToeNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            pred = model(xb)
            pred_labels = pred.argmax(dim=1)
            correct += (pred_labels == yb).sum().item()
            total += yb.size(0)
    acc = correct / total * 100
    print(f"Epoch {epoch+1}: Train Loss = {total_loss:.4f}, Val Accuracy = {acc:.2f}%")


Epoch 1: Train Loss = 2668.2532, Val Accuracy = 32.97%
Epoch 2: Train Loss = 2407.5602, Val Accuracy = 33.99%
Epoch 3: Train Loss = 2364.7139, Val Accuracy = 34.55%
Epoch 4: Train Loss = 2338.9917, Val Accuracy = 35.11%
Epoch 5: Train Loss = 2322.0725, Val Accuracy = 34.89%
Epoch 6: Train Loss = 2308.7576, Val Accuracy = 35.22%
Epoch 7: Train Loss = 2297.1413, Val Accuracy = 35.33%
Epoch 8: Train Loss = 2289.0666, Val Accuracy = 35.27%
Epoch 9: Train Loss = 2283.4496, Val Accuracy = 35.19%
Epoch 10: Train Loss = 2278.1972, Val Accuracy = 35.54%


In [ ]:
torch.save(model.state_dict(), 'tictactoe_model.pth')

In [ ]:
from google.colab import files
files.download('tictactoe_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>